# Building a Sepsis Screening Protocol

This notebook walks through building a multi-step sepsis screening workflow using MOISSCode modules. We combine clinical scoring, lab interpretation, dosing, and FHIR export.

In [ ]:
from moisscode import StandardLibrary, Patient

lib = StandardLibrary()

## Step 1: Create the Patient

In [ ]:
patient = Patient(
    name='John Doe',
    age=72,
    weight=80.0,
    bp=82,
    hr=118,
    rr=26,
    temp=39.1,
    spo2=91,
    gcs=13,
    lactate=4.8,
    creatinine=2.4,
    sex='M'
)
print(f'Patient: {patient.name}, Age: {patient.age}, Sex: {patient.sex}')

## Step 2: Clinical Scoring

In [ ]:
qsofa = lib.scores.qsofa(patient)
sofa = lib.scores.sofa(patient)
news = lib.scores.news2(patient)

print(f'qSOFA: {qsofa} (>= 2 = high risk)')
print(f'SOFA: {sofa} (>= 2 = organ dysfunction)')
print(f'NEWS2: {news} (>= 7 = clinical emergency)')

## Step 3: Lab Assessment

In [ ]:
# Check key labs
wbc = lib.lab.interpret('WBC', 22.1)
lactate_result = lib.lab.interpret('Lactate', patient.lactate)
creatinine_result = lib.lab.interpret('Creatinine', patient.creatinine)

for r in [wbc, lactate_result, creatinine_result]:
    critical = ' *** CRITICAL ***' if r.get('is_critical') else ''
    print(f'{r["test"]}: {r["value"]} {r["unit"]} [{r["status"]}]{critical}')

# Renal function
gfr = lib.lab.gfr(creatinine=patient.creatinine, age=patient.age, sex=patient.sex)
print(f'eGFR: {gfr["eGFR"]} {gfr["unit"]} ({gfr["stage"]})']

## Step 4: Antibiotic Dosing with Renal Adjustment

In [ ]:
# Calculate dose
dose = lib.pk.calculate_dose('Vancomycin', weight_kg=patient.weight)
print(f'Standard dose: {dose}')

# Renal adjustment
adjustment = lib.pk.renal_adjust('Vancomycin', gfr=gfr['eGFR'])
print(f'Renal adjustment factor: {adjustment["adjustment_factor"]}')
print(f'Recommendation: {adjustment["recommendation"]}')

# Check interactions
interactions = lib.pk.check_interactions('Vancomycin', current_drugs=['Gentamicin', 'Furosemide'])
print(f'Interactions: {interactions["max_severity"]}')

## Step 5: Intervention Timing Classification

In [ ]:
# How timely is our Vancomycin administration?
classification = lib.moiss.classify(t_crit_min=45, drug_name='Vancomycin')
print(f'MOISS classification: {classification}')

## Step 6: Generate FHIR Bundle

In [ ]:
bundle = lib.fhir.to_fhir(patient)
print(f'FHIR Resource Type: {bundle["resourceType"]}')
for key, value in bundle.items():
    if key != 'resourceType':
        print(f'  {key}: {value}')

## Summary

In this notebook we used 6 MOISSCode modules in a single workflow:

1. **ClinicalScores** for qSOFA, SOFA, NEWS2
2. **LabEngine** for WBC, lactate, creatinine interpretation + GFR
3. **PharmacokineticEngine** for dosing + renal adjustment + interactions
4. **MOISS_Classifier** for intervention timing
5. **FHIRBridge** for structured data export
6. **Patient** dataclass tying everything together

---
*MOISSCode is for research and educational use only.*